---
## AI-Model Testing
by Tobia Ippolito
### Contents:
- [Imports](#0.0)
- [10-fold-Crossvalidation]()
- [SVC]()
- [KNN]()
- [RandomForest]()
- [VotingClassifier]()
<br>
<br>
---
### <a name='0.0'>Imports</a>

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../src')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

import anoog
from anoog.model import MODELS
from anoog.model import evaluate_model_with_cross_validation
from anoog.model import evaluate_model
from anoog.model import get_most_important_features

---
### <a name='1.0'>10-fold-Crossvalidation</a>

In [3]:
data = df = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False,
                        normalize=True)
data.tail()

,audio_min,audio_max,audio_mean,audio_median,audio_std,current_min,current_max,current_mean,current_median,current_std,...,voltage_mean,voltage_median,voltage_std,Time,Resistance,Power,Work,mAh,Wh,y
45,0.499970,0.524153,0.546429,0.486107,0.498624,0.981361,0.794324,0.099471,0.051765,0.403033,...,0.262847,0.322230,0.370096,0.629174,0.098147,0.100821,0.280750,0.099471,0.100821,1.0
46,0.501927,0.519836,0.631510,0.485915,0.462201,0.950122,0.780712,0.106452,0.104558,0.384292,...,0.254285,0.310874,0.346838,0.521968,0.105264,0.107660,0.118180,0.106452,0.107660,1.0
47,0.703289,0.393621,0.512861,0.487407,0.263667,0.420836,0.000000,0.000000,0.031463,0.000000,...,0.274758,0.294388,0.000000,0.760984,0.000000,0.000000,0.166171,0.000000,0.000000,1.0
48,0.502029,0.513907,0.577089,0.486803,0.431190,0.420836,0.876737,0.173159,0.130077,0.436350,...,0.196933,0.256548,0.390586,0.520211,0.172791,0.173515,0.288815,0.173159,0.173515,1.0
49,0.792630,0.469935,0.792698,0.454811,0.339974,0.420836,0.862295,0.216525,0.194656,0.407967,...,0.157877,0.198933,0.368864,0.534271,0.216968,0.216049,0.429380,0.216525,0.216049,1.0


In [4]:
X,y = anoog.io.X_y_split(data)
X.head()

,audio_min,audio_max,audio_mean,audio_median,audio_std,current_min,current_max,current_mean,current_median,current_std,...,voltage_max,voltage_mean,voltage_median,voltage_std,Time,Resistance,Power,Work,mAh,Wh
0,0.489226,0.539696,0.620803,0.488007,0.589802,0.841672,0.903051,0.081916,0.047550,0.342350,...,1.000000,1.000000,1.000000,0.387944,0.984183,0.061104,0.104178,0.892324,0.081916,0.104178
1,0.948940,0.078485,0.737151,0.497329,0.000000,0.912990,0.870149,0.478445,0.668963,0.707884,...,0.894985,0.701696,0.586562,0.730675,0.356766,0.450447,0.507766,0.700566,0.478445,0.507766
2,0.554606,0.519374,0.720443,0.496879,0.187601,0.841672,0.971043,0.673538,0.751618,0.726680,...,0.865587,0.585654,0.542533,0.742611,0.191564,0.644603,0.703628,0.554519,0.673538,0.703628
3,0.500641,0.526755,0.598239,0.488965,0.406655,0.841672,0.902161,0.093400,0.079668,0.351717,...,0.851916,0.882752,0.883772,0.364501,0.736380,0.075100,0.112851,0.500696,0.093400,0.112851
4,0.000000,0.532166,0.317739,0.452649,0.614912,0.420836,0.807761,0.452521,0.502235,0.487880,...,0.836230,0.677153,0.600962,0.506825,0.353251,0.426599,0.479638,0.630015,0.452521,0.479638


In [5]:
evaluate_model_with_cross_validation(X, y, model=MODELS.RANDOM_FOREST)

array([0.7, 0.8, 0.9, 1. , 0.6])

In [6]:
evaluate_model_with_cross_validation(X, y, model=RandomForestClassifier())

array([0.7, 0.8, 0.9, 1. , 0.8])

In [7]:
evaluate_model_with_cross_validation(X, y, model=MODELS.SVM)

array([1. , 0.8, 0.9, 1. , 0.8])

In [8]:
evaluate_model_with_cross_validation(X, y, model=MODELS.KNN)

array([0.7, 0.6, 0.5, 0.9, 0.8])

In [9]:
evaluate_model_with_cross_validation(X, y, model=MODELS.LOGISTIC_REGRESSION)

array([0.8, 0.8, 0.8, 1. , 0.8])

In [10]:
evaluate_model_with_cross_validation(X, y, model=MODELS.ADA_BOOST)

array([0.6, 1. , 1. , 0.7, 0.5])

In [11]:
evaluate_model_with_cross_validation(X, y, model=MODELS.NAIVE_BAYES)

array([0.8, 0.7, 0.7, 0.9, 0.8])

In [12]:
evaluate_model_with_cross_validation(X, y, model=MODELS.VOTING_CLASSIFIER)

array([0.7, 0.7, 1. , 1. , 0.8])

---
### <a name='2.0'>**S**upport **V**ector **C**lassifier</a>
Good with high dimensions, normlizing because the distance

get best params

In [13]:
data = df = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False,
                        normalize=True)

In [14]:
X,y = anoog.io.X_y_split(data)

In [15]:
param_grid = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
              'probability':[True, False], 'decision_function_shape':['ovo', 'ovr'],
              'C':[0.001,0.1,0.2,0.5,1.0,1.5,2.0,2.5,3.0]  }
grid_searcher = GridSearchCV(SVC(), param_grid)
grid_searcher.fit(X, y)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.001, 0.1, 0.2, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0],
                         'decision_function_shape': ['ovo', 'ovr'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'probability': [True, False]})

In [16]:
grid_searcher.best_params_

{'C': 0.1,
 'decision_function_shape': 'ovo',
 'kernel': 'poly',
 'probability': True}

Try best params

In [17]:
train, test = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=True,
                        test_size=0.3,
                        normalize=True)

../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]
../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]


In [18]:
X_train, y_train = anoog.io.X_y_split(train)
X_test, y_test = anoog.io.X_y_split(test)

In [19]:
model = SVC(C= 0.1, decision_function_shape='ovo', kernel='poly', probability=True)
model = model.fit(X_train, y_train)

In [20]:
acc, con = evaluate_model(model, X_test, y_test)

print(f"#### RESULTS ####\n\nAccuracy: ", acc)
print("--------\nConfusion Matrix:")
print(con, "\n")

#### RESULTS ####

Accuracy:  93.33333333333333
--------
Confusion Matrix:
[[9 1]
 [0 5]] 



check with 10-fold-Crossvalidation

In [21]:
evaluate_model_with_cross_validation(X, y, model=MODELS.SVM)

array([1. , 0.8, 0.9, 1. , 0.8])

---
### <a name='3.0'>**K**-**N**earest**N**eighbors-Classifier</a>
Bad with high dimensionality, needs a normalization.

In [22]:
data = df = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False,
                        normalize=True)

In [23]:
X,y = anoog.io.X_y_split(data)
X.tail()

,audio_min,audio_max,audio_mean,audio_median,audio_std,current_min,current_max,current_mean,current_median,current_std,...,voltage_max,voltage_mean,voltage_median,voltage_std,Time,Resistance,Power,Work,mAh,Wh
45,0.499970,0.524153,0.546429,0.486107,0.498624,0.981361,0.794324,0.099471,0.051765,0.403033,...,0.071831,0.262847,0.322230,0.370096,0.629174,0.098147,0.100821,0.280750,0.099471,0.100821
46,0.501927,0.519836,0.631510,0.485915,0.462201,0.950122,0.780712,0.106452,0.104558,0.384292,...,0.061311,0.254285,0.310874,0.346838,0.521968,0.105264,0.107660,0.118180,0.106452,0.107660
47,0.703289,0.393621,0.512861,0.487407,0.263667,0.420836,0.000000,0.000000,0.031463,0.000000,...,0.042432,0.274758,0.294388,0.000000,0.760984,0.000000,0.000000,0.166171,0.000000,0.000000
48,0.502029,0.513907,0.577089,0.486803,0.431190,0.420836,0.876737,0.173159,0.130077,0.436350,...,0.042432,0.196933,0.256548,0.390586,0.520211,0.172791,0.173515,0.288815,0.173159,0.173515
49,0.792630,0.469935,0.792698,0.454811,0.339974,0.420836,0.862295,0.216525,0.194656,0.407967,...,0.000000,0.157877,0.198933,0.368864,0.534271,0.216968,0.216049,0.429380,0.216525,0.216049


In [24]:
X = anoog.io.model_based_feature_selection(X, y, n=3)
X.tail()

,voltage_max,voltage_mean,voltage_median,voltage_std,current_max
45,0.071831,0.262847,0.322230,0.370096,0.794324
46,0.061311,0.254285,0.310874,0.346838,0.780712
47,0.042432,0.274758,0.294388,0.000000,0.000000
48,0.042432,0.196933,0.256548,0.390586,0.876737
49,0.000000,0.157877,0.198933,0.368864,0.862295


In [25]:
param_grid = {'n_neighbors':[1,2,3,4,5,6,7,8,9],
              'weights':['uniform', 'distance'], 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
              'metric':['minkowski', 'euclidean', 'manhattan', 'chebyshev']  }
grid_searcher = GridSearchCV(KNN(), param_grid)
grid_searcher.fit(X, y)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski', 'euclidean', 'manhattan',
                                    'chebyshev'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'weights': ['uniform', 'distance']})

In [26]:
grid_searcher.best_params_

{'algorithm': 'auto',
 'metric': 'chebyshev',
 'n_neighbors': 1,
 'weights': 'uniform'}

test best params

In [27]:
train, test = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=True,
                        test_size=0.3,
                        normalize=True)

../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]
../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]


In [28]:
X_train, y_train = anoog.io.X_y_split(train)
X_test, y_test = anoog.io.X_y_split(test)

In [29]:
model = KNN(algorithm='auto', metric='chebyshev', n_neighbors=6, weights='uniform')
model = model.fit(X_train, y_train)

In [30]:
acc, con = evaluate_model(model, X_test, y_test)

print(f"#### RESULTS ####\n\nAccuracy: ", acc)
print("--------\nConfusion Matrix:")
print(con, "\n")

#### RESULTS ####

Accuracy:  60.0
--------
Confusion Matrix:
[[5 0]
 [6 4]] 



check with 10-fold-Crossvalidation

In [31]:
evaluate_model_with_cross_validation(X, y, model=MODELS.KNN)

array([0.6, 0.8, 0.8, 1. , 0.8])

---
### <a name='4.0'>Voting Classifier</a>

find best params

In [32]:
data = df = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False,
                        normalize=True)

In [33]:
X,y = anoog.io.X_y_split(data)
X.tail()

,audio_min,audio_max,audio_mean,audio_median,audio_std,current_min,current_max,current_mean,current_median,current_std,...,voltage_max,voltage_mean,voltage_median,voltage_std,Time,Resistance,Power,Work,mAh,Wh
45,0.499970,0.524153,0.546429,0.486107,0.498624,0.981361,0.794324,0.099471,0.051765,0.403033,...,0.071831,0.262847,0.322230,0.370096,0.629174,0.098147,0.100821,0.280750,0.099471,0.100821
46,0.501927,0.519836,0.631510,0.485915,0.462201,0.950122,0.780712,0.106452,0.104558,0.384292,...,0.061311,0.254285,0.310874,0.346838,0.521968,0.105264,0.107660,0.118180,0.106452,0.107660
47,0.703289,0.393621,0.512861,0.487407,0.263667,0.420836,0.000000,0.000000,0.031463,0.000000,...,0.042432,0.274758,0.294388,0.000000,0.760984,0.000000,0.000000,0.166171,0.000000,0.000000
48,0.502029,0.513907,0.577089,0.486803,0.431190,0.420836,0.876737,0.173159,0.130077,0.436350,...,0.042432,0.196933,0.256548,0.390586,0.520211,0.172791,0.173515,0.288815,0.173159,0.173515
49,0.792630,0.469935,0.792698,0.454811,0.339974,0.420836,0.862295,0.216525,0.194656,0.407967,...,0.000000,0.157877,0.198933,0.368864,0.534271,0.216968,0.216049,0.429380,0.216525,0.216049


test best params

In [34]:
train, test = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=True,
                        test_size=0.3,
                        normalize=True)

../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]
../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]


In [35]:
X_train, y_train = anoog.io.X_y_split(train)
X_test, y_test = anoog.io.X_y_split(test)

In [36]:
estimators_1 = [('SVC', SVC(C= 0.1, decision_function_shape='ovo', kernel='poly', probability=True)), 
                      ('RandomForest', RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=5))]

estimators_2 = [('SVC', SVC(C= 0.1, decision_function_shape='ovo', kernel='poly', probability=True)), 
                ('RandomForest', RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=5)),
                ('RandomForest2', RandomForestClassifier())]

estimators_3 = [('SVC', SVC(C= 0.1, decision_function_shape='ovo', kernel='poly', probability=True)), 
                ('RandomForest', RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=5)),
                ('KNN', KNN(algorithm='auto', metric='chebyshev', n_neighbors=6, weights='uniform'))]

model = VotingClassifier(estimators=estimators_3, voting='hard')
model = model.fit(X_train, y_train)

In [37]:
acc, con = evaluate_model(model, X_test, y_test)

print(f"#### RESULTS ####\n\nAccuracy: ", acc)
print("--------\nConfusion Matrix:")
print(con, "\n")

#### RESULTS ####

Accuracy:  80.0
--------
Confusion Matrix:
[[6 0]
 [3 6]] 



check with 10-fold-Crossvalidation

In [38]:
evaluate_model_with_cross_validation(X, y, model=MODELS.VOTING_CLASSIFIER)

array([0.7, 0.7, 1. , 1. , 0.8])

---
### <a name='5.0'>Random Forest</a>

find best params

In [39]:
data = df = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False,
                        normalize=False)

In [40]:
X,y = anoog.io.X_y_split(data)
X.tail()

,audio_min,audio_max,audio_mean,audio_median,audio_std,current_min,current_max,current_mean,current_median,current_std,...,voltage_max,voltage_mean,voltage_median,voltage_std,Time,Resistance,Power,Work,mAh,Wh
45,-3.813901,3.485494,-0.005958,-0.000307,0.923199,-0.841899,30.302135,1.953009,1.898246,2.931288,...,19.666992,19.236397,19.249631,0.376643,5.76,0.101527,37.568856,216.396609,32.550150,626.147595
46,-3.805070,3.467371,-0.002143,-0.000328,0.879257,-0.847336,29.887917,1.977015,2.087429,2.853247,...,19.658431,19.227930,19.237225,0.363789,5.15,0.102820,38.013911,195.771642,32.950254,633.565184
47,-2.896698,2.937443,-0.007464,-0.000168,0.639745,-0.939453,6.131235,1.610943,1.825491,1.252940,...,19.643066,19.248177,19.219212,0.172090,6.51,0.083693,31.007708,201.860179,26.849043,516.795133
48,-3.804612,3.442479,-0.004583,-0.000233,0.841846,-0.939453,32.809911,2.206411,2.178879,3.070032,...,19.643066,19.171210,19.177871,0.387968,5.14,0.115090,42.299573,217.419805,36.773520,704.992883
49,-2.493667,3.257855,0.005086,-0.003672,0.731803,-0.939453,32.370458,2.355542,2.410300,2.951833,...,19.608533,19.132585,19.114923,0.375963,5.22,0.123117,45.067612,235.252934,39.259037,751.126866


In [41]:
param_grid = {'n_estimators':[10, 25, 50, 75, 100, 125, 150],
              'criterion':['gini', 'entropy'], 'min_samples_split':[2,3,4,5]}
grid_searcher = GridSearchCV(RandomForestClassifier(), param_grid)
grid_searcher.fit(X, y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [10, 25, 50, 75, 100, 125, 150]})

In [42]:
grid_searcher.best_params_

{'criterion': 'entropy', 'min_samples_split': 4, 'n_estimators': 10}

Test best params

In [43]:
train, test = anoog.io.load_data("../../data/2021-11-09",
                        ['tippolit', 'vkorzev'],
                        extraction=anoog.io.extraction_mode.MANUEL,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=True,
                        test_size=0.3)
train.tail()

../../src\anoog\io\data_io.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataset['y'] = data[1]


,audio_min,audio_max,audio_mean,audio_median,audio_std,current_min,current_max,current_mean,current_median,current_std,...,voltage_mean,voltage_median,voltage_std,Time,Resistance,Power,Work,mAh,Wh,y
13,-5.712891,3.422858,-0.028224,-0.014136,1.235233,-0.850208,30.270541,3.522206,4.423537,4.191823,...,19.427482,19.281837,0.555071,3.32,0.181300,68.427596,227.179618,58.703435,1140.459929,0
21,-3.687213,3.800531,0.012627,-0.011499,1.247428,-0.866211,33.876846,3.150045,3.957732,4.098388,...,19.369811,19.263940,0.525924,3.93,0.162627,61.015780,239.792016,52.500752,1016.929670,0
47,-2.896698,2.937443,-0.007464,-0.000168,0.639745,-0.939453,6.131235,1.610943,1.825491,1.252940,...,19.248177,19.219212,0.172090,6.51,0.083693,31.007708,201.860179,26.849043,516.795133,1
16,-3.841994,3.481794,-0.024132,-0.052576,0.993623,-0.939453,36.434681,4.928060,5.296288,5.371777,...,19.183455,19.130656,0.724793,2.24,0.256891,94.537221,211.763376,82.134337,1575.620355,0
42,-1.558192,1.284775,0.000740,0.001490,0.374571,-0.861697,32.737817,3.043812,4.087125,3.661341,...,19.139812,18.982462,0.468965,4.23,0.159030,58.257982,246.431262,50.730193,970.966360,1


In [44]:
X_train, y_train = anoog.io.X_y_split(train)
X_test, y_test = anoog.io.X_y_split(test)

Train Model

In [45]:
model = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_split=5)
# Fit on training data
model = model.fit(X_train, y_train)

In [46]:
get_most_important_features(model, X_train, n=len(X_train.columns))

1. voltage_max (0.3313771600239712)
2. voltage_mean (0.1619196069485143)
3. voltage_median (0.08862922022306784)
4. audio_median (0.051892546001945365)
5. Power (0.04729223126857127)
6. current_min (0.03383066291927245)
7. Wh (0.02902338796624733)
8. mAh (0.028844096193576748)
9. current_max (0.027296704919449733)
10. audio_max (0.027186360225878493)
11. Resistance (0.02597141614659174)
12. current_mean (0.022964903540157983)
13. Time (0.0223241583478747)
14. current_median (0.02052799240823404)
15. voltage_min (0.01727238095964909)
16. voltage_std (0.016941895506621153)
17. audio_std (0.01437551352809926)
18. audio_min (0.01348006194214582)
19. audio_mean (0.008282575280244876)
20. current_std (0.006228636191520812)
21. Work (0.004338489458365784)


Evaluate

In [47]:
acc, con = evaluate_model(model, X_test, y_test)

print(f"#### RESULTS ####\n\nAccuracy: ", acc)
print("--------\nConfusion Matrix:")
print(con, "\n")

#### RESULTS ####

Accuracy:  86.66666666666667
--------
Confusion Matrix:
[[ 3  0]
 [ 2 10]] 



---